In [1]:
import numpy as np
import os
os.environ["KERAS_BACKEND"] = "plaidml.keras.backend"
from preprocessing import *

import keras
from keras import applications
from keras.models import Sequential, Model
from keras.layers import Conv2D, MaxPooling2D, LSTM, Reshape, Permute
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.layers.merge import concatenate
#suppress warnings
import warnings
warnings.filterwarnings('ignore')

Using plaidml.keras.backend backend.


In [ ]:
#data dimension will be: #bag * #frame * 244 * 244 * 3
#simple implementation of CNN>LSTM without multires


In [2]:
testPath='/Users/lingfengli/Desktop/SIR/CRNN/ResizeBinnedToClips6F/Test'
trainPath='/Users/lingfengli/Desktop/SIR/CRNN/ResizeBinnedToClips6F/Training'
validPath='/Users/lingfengli/Desktop/SIR/CRNN/ResizeBinnedToClips6F/Validation'
testl,testd=prepdata(testPath)
trainl,traind=prepdata(trainPath)
validl,validd=prepdata(validPath)

In [ ]:
len(testl)

In [22]:
#spawn #instance of CNN = #timestep of RNN
model = applications.inception_v3.InceptionV3(weights = "imagenet", include_top=False, input_shape = ( 244, 244, 3))
model2= applications.inception_v3.InceptionV3(weights = "imagenet", include_top=False, input_shape = ( 244, 244, 3))

In [ ]:
len(model.layers)

In [ ]:
type(intermediateOut)

In [ ]:
for layer in model.layers[:300]:
    layer.trainable = False
for layer in model.layers[300:]:
    layer.trainable=True

In [ ]:
intermediateOut = model.output
intermediateOut2= model2.output
intermediateOut2 = Flatten()(intermediateOut2)
intermediateOut = Flatten()(intermediateOut)


In [ ]:
intermediateOut = Reshape((1,intermediateOut.shape.dims[1]))(intermediateOut)
intermediateOut2= Reshape((1,intermediateOut2.shape.dims[1]))(intermediateOut2)

In [ ]:
intermediateOut2.shape

In [ ]:
x=concatenate([intermediateOut, intermediateOut2], axis=1)

In [ ]:
#x dimension: #batchsize = 1, #timestep = # instance spawned, #73728
x.shape

In [23]:
intermediateOut = model.output
intermediateOut2= model2.output
model2.get_layer("mixed0").name="layer_x"
model2.get_layer("mixed1").name="layer_x10"
model2.get_layer("mixed2").name="layer_x1"
model2.get_layer("mixed3").name="layer_x2"
model2.get_layer("mixed4").name="layer_x3"
model2.get_layer("mixed5").name="layer_x4"
model2.get_layer("mixed6").name="layer_x5"
model2.get_layer("mixed7").name="layer_x6"
model2.get_layer("mixed8").name="layer_x7"
model2.get_layer("mixed9").name="layer_x8"
model2.get_layer("mixed9_0").name="layer_x81"
model2.get_layer("mixed9_1").name="layer_x82"
model2.get_layer("mixed10").name="layer_x9"
intermediateOut = Flatten()(intermediateOut)
intermediateOut2 = Flatten()(intermediateOut2)
############################
#How to manage dimension   #
#and merge models together?#
############################
intermediateOut = Reshape((1,intermediateOut.shape.dims[1]))(intermediateOut)
intermediateOut2= Reshape((1,intermediateOut2.shape.dims[1]))(intermediateOut2)
x=concatenate([intermediateOut, intermediateOut2], axis=1)

ii=model.input
iii=model.input

intermediateOut = LSTM(128, return_sequences = True)(x)
intermediateOut = BatchNormalization()(intermediateOut)
intermediateOut = Dropout(0.4)(intermediateOut)
intermediateOut = LSTM(64, return_sequences = True)(intermediateOut)
intermediateOut = BatchNormalization()(intermediateOut)
intermediateOut = Dropout(0.4)(intermediateOut)
intermediateOut = LSTM(32)(intermediateOut)
intermediateOut = BatchNormalization()(intermediateOut)
intermediateOut = Dropout(0.4)(intermediateOut)
intermediateOut = Dense(32, activation='relu')(intermediateOut)
intermediateOut = BatchNormalization()(intermediateOut)
intermediateOut = Dropout(0.4)(intermediateOut)
prediction = Dense(25, activation='softmax')(intermediateOut)
optmr = keras.optimizers.adam(lr=0.001, decay = 1e-5)
model_final = Model(input = [ii,iii], output = prediction)
model_final.compile(loss = "categorical_crossentropy", optimizer = optmr, metrics=["accuracy"])

ValueError: The list of inputs passed to the model is redundant. All inputs should only appear once. Found: [<tile.Value input_9 Placeholder FLOAT32(<tile.Value SymbolicDim UINT64()>, 244, 244, 3)>, <tile.Value input_9 Placeholder FLOAT32(<tile.Value SymbolicDim UINT64()>, 244, 244, 3)>]

In [ ]:
optmr = keras.optimizers.adam(lr=0.001, decay = 1e-5)
model_final = Model(input = [model.input,model2.input], output = prediction)
model_final.compile(loss = "categorical_crossentropy", optimizer = optmr, metrics=["accuracy"])

In [18]:
model_final.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 244, 244, 3)  0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, 244, 244, 3)  0                                            
__________________________________________________________________________________________________
conv2d_377 (Conv2D)             (None, 121, 121, 32) 864         input_5[0][0]                    
__________________________________________________________________________________________________
conv2d_471 (Conv2D)             (None, 121, 121, 32) 864         input_6[0][0]                    
__________________________________________________________________________________________________
batch_norm